In [22]:
# To change the 

import pandas as pd
import numpy as np
import datetime
import os
import glob
import logging


def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
logging.basicConfig(filename="Daily_Data_log.log", level=logging.INFO)

logging.info("Start at "+str(datetime.datetime.now()))

In [23]:
today_date=datetime.datetime.now().date()
last_week_Saturday=today_date-datetime.timedelta(days=(today_date.weekday()+2))
last_week_Saturday_str=str(last_week_Saturday)
last_year_Saturday_str=str(today_date-datetime.timedelta(days=(today_date.weekday()+2+52*7)))


In [24]:
write_folder="/home/jian/Projects/Big_Lots/celery_BL_daily_data/output/output_"+last_week_Saturday_str+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

In [25]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders="/home/jian/BigLots/MediaStorm_"+last_week_Saturday_str+"/"
daily_data_recent=[x for x in list(recursive_file_gen(possible_recent_folders)) if ("aily" in x) & (".txt" in x)]

archived_folders_2018="/home/jian/BigLots/2018_by_weeks/MediaStorm_"+last_week_Saturday_str+"/"
daily_data_2018_achived=[x for x in list(recursive_file_gen(archived_folders_2018)) if ("aily" in x) & (".txt" in x)]

archived_folders_2019="/home/jian/BigLots/2019_by_weeks/MediaStorm_"+last_week_Saturday_str+"/"
daily_data_2019_achived=[x for x in list(recursive_file_gen(archived_folders_2019)) if ("aily" in x) & (".txt" in x)]

daily_data_list=daily_data_recent+daily_data_2018_achived+daily_data_2019_achived
if len(daily_data_list)!=1:
    logging.info("Error of Daily Sales Data Path")
else:
    daily_data_path_this_year=daily_data_list[0]

In [26]:
daily_data_list_hist=glob.glob("/home/jian/BigLots/hist_daily_data/*.txt")
daily_data_list_hist=[x for x in daily_data_list_hist if (last_year_Saturday_str in x)&("DailySales" in x)]
if len(daily_data_list_hist)!=1:
    logging.info("Error of Daily Sales Data Path")
else:
    daily_data_path_last_year=daily_data_list_hist[0]

In [27]:
Df_2018Q3_Quadrant=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/BL_Excutive_Dashboard/Excel_BL_data_for_2018Q3_Quadrants_JL_2018-12-04_Q2Scale.xlsx",
                                 dype=str,sheetname="2018_Q3_Quadrants",usecols=['location_id','Quadrant in Graph 1-2'])
Df_2018Q3_Quadrant=Df_2018Q3_Quadrant[['location_id','Quadrant in Graph 1-2']].rename(columns={"Quadrant in Graph 1-2":"Quadrant"})
Df_2018Q3_Quadrant['location_id']=Df_2018Q3_Quadrant['location_id'].astype(str)
Df_2018Q3_Quadrant.shape

(1363, 2)

In [28]:
def count_unique_id(x):
    return len(set(x))


In [29]:
def agg_daily_data_by_store(file_path_daily):
    df=pd.read_table(file_path_daily,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    df=df[df['location_id']!="6990"] # 6990 removed
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df['Reward_Level']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    
    print("Total_Sales:",df['subclass_transaction_amt'].sum())
    df_sales=df.groupby(['location_id','Reward_Level'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"Sales"})
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','Reward_Level']].drop_duplicates().groupby(['location_id','Reward_Level'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"Transactions"})
    
    df=df[df['Reward_Level']=="Rewards"]
    df_ids=df.groupby(['location_id','Reward_Level'])['customer_id_hashed'].agg(count_unique_id).to_frame().reset_index().rename(columns={"customer_id_hashed":"shopped_unique_ids_in_the_week"})
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'Reward_Level'],how="outer")
    df=pd.merge(df,df_ids,on=["location_id",'Reward_Level'],how="outer")
    
    return df

In [30]:
df_daily_this_year_by_store=agg_daily_data_by_store(daily_data_path_this_year)
df_daily_last_year_by_store=agg_daily_data_by_store(daily_data_path_last_year)


Total_Sales: 80218757.31
Total_Sales: 67199440.68


In [ ]:
last_week_daily_for_Bing=pd.read_table(daily_data_path_this_year,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])



